# ChessML Netz

This neural network based on PyTorch will try to classifiy chess pieces from photos taken from the top of a chessboard.

In [ ]:
import os
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import random

from torchvision import transforms
from PIL import Image
from os import listdir

## Defining all possible classes

bb = Black Bishop
bk = Black King
bn = Black Knight
bp = Black Pawn
bq = Black Queen
br = Black Rook
...

In [ ]:
CLASSES = ["bb", "bk", "bn", "bp", "bq", "br", "wb", "wk", "wn", "wp", "wq", "wr", "empty"]

## Define a normalization function for the analyzed data

The values are recommended by a PyTorch tutorial (https://youtu.be/32lHVbT09h8).

In [ ]:
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

transform = transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor(),
    normalize
])

## Defining the Neural Network

In [ ]:
class ChessNet(nn.Module):
    def __init__(self):
        super(ChessNet, self).__init__()
        # Defining the convolutional layers of the net
        self.conv1 = nn.Conv2d(3, 6, kernel_size=3)
        self.conv2 = nn.Conv2d(6, 12, kernel_size=3)
        self.conv3 = nn.Conv2d(12, 24, kernel_size=3)
        self.conv4 = nn.Conv2d(24, 48, kernel_size=3)
        self.conv5 = nn.Conv2d(48, 96, kernel_size=3)

        # Defining dropout layer to prevent from memorizing
        self.dropout1 = nn.Dropout(p=0.3)
        self.dropout2 = nn.Dropout(p=0.3)

        # Defining the fully connected layers of the net
        self.fc1 = nn.Linear(384, 64)  # 96*2*2 = 384
        self.fc2 = nn.Linear(64, 13)

    def forward(self, x):
        x = self.conv1(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)  # Relu because it's famous

        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)

        x = self.conv3(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)

        x = self.conv4(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)

        x = self.conv5(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)

        x = self.dropout1(x)

        x = x.view(-1, 384)  # Convert 2d data to 1d

        x = self.fc1(x)
        x = self.dropout2(x)
        x = F.relu(x)

        x = self.fc2(x)

        return torch.sigmoid(x)


## Training

In [ ]:
def train(model, epoch, train_data, optimizer):
    model.train()
    batch_id = 0

    for data, target in train_data:
        target = torch.Tensor(target)
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()

        optimizer.zero_grad()
        out = model(data)
        criterion = F.binary_cross_entropy
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_id * len(data), len(train_data) * len(data), 100. * batch_id / len(train_data), loss.item()
        ))
        batch_id = batch_id + 1


## Evaluation

In [ ]:
def evaluate(model, eval_data):
    model.eval()
    loss = 0
    correct = 0
    for data, target in eval_data:
        target = torch.Tensor(target)
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
            
        out = model(data)
        criterion = F.binary_cross_entropy
        loss += criterion(out, target, size_average=False).item()
        prediction = out.data.max(1, keepdim=True)[1]
        if torch.cuda.is_available():
            correct += prediction.eq(target.view_as(prediction)).cpu().sum()
        else:
            correct += prediction.eq(target.view_as(prediction)).sum()
            
    loss = loss / len(eval_data)
    print("###################################")
    print("Average loss:", loss)
    print("Accuracy:", 100. * correct / len(eval_data))
    print("###################################")


## Testing

The test function is not really necessary. It is just for a better understanding and eventually testing the neural network manually.

In [ ]:
def test(model, path):
    model.eval()
    files = os.listdir(path)
    file = random.choice(files)
    img = Image.open(os.path.join(path, file))
    img_eval_tensor = transform(img)
    img_eval_tensor.unsqueeze_(0)
    if torch.cuda.is_available():
        img_eval_tensor = img_eval_tensor.cuda()
    out = model(img_eval_tensor)
    print(out.data.max(1, keepdim=True))
    img.show()
    x = input("")


## Helper functions

At first all available files are added to a file list. In addition the label of the file will be generated and added to the list.

In [ ]:
def read_files(path):
    file_list = []
    for d in os.listdir(path):
        if os.path.isdir(os.path.join(path, d)):
            for f in os.listdir(os.path.join(path, d)):
                file_list.append([os.path.join(path, d, f), hot_encode_label(d)])
    return file_list


Generating tensors from the file list, we just created. All files get mixed up randomly.

In [ ]:
def generate_tensors(file_list, batch_size):
    train_data_list = []
    train_data = []
    label_list = []
    files_count = len(file_list)
    for i in range(files_count):
        file = random.choice(file_list)
        file_list.remove(file)
        img = Image.open(file[0])  # Index 0: filename, Index 1: label
        img_tensor = transform(img)
        train_data_list.append(img_tensor)
        label_list.append(file[1])
        if len(train_data_list) >= batch_size:
            train_data.append((torch.stack(train_data_list), label_list))
            train_data_list = []
            label_list = []
            print("Loaded batch", len(train_data), "of", int(files_count / batch_size))
            print("Percentage Done:", len(train_data) / int(files_count / batch_size) * 100, "%")
            if len(train_data) >= 1:
                break
    return train_data


The label will be hot encoded, since there is only one possible class for each tile of the chessboard. 
This means that for example a black knight will be labeled by the vector: 

>\[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0\]

In [ ]:
def hot_encode_label(label):
    index = CLASSES.index(label)
    vector = np.zeros(len(CLASSES), np.int8)
    vector[index] = 1
    return vector


## Main

Training and evaluating the ChessNet

In [ ]:
def main():
    file_list = read_files("data/train_augmented")
    train_data = generate_tensors(file_list, 64)

    eval_files = read_files("data/eval_augmented")
    eval_data  = generate_tensors(file_list, 64)
    
    model = ChessNet()
    # Activate cuda support if available
    if torch.cuda.is_available():
        model = model.cuda()

    # Check if model is already available
    # if os.path.isfile("model/chess-net.pt"):
    #     model = torch.load("model/chess-net.pt")

    # Defining the optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    # optimizer = optim.RMSprop(model.parameters(), lr=1e-3)

    for epoch in range(1,30):
        train(model, epoch, train_data, optimizer)
        evaluate(model, eval_data)
        # test(model, "data/train_augmented/bn")
        torch.save(model, "model/chess-net.pt")
    print("Training of the neuroal network done.")


In [ ]:
if __name__ == "__main__":
    main()
